In [1]:
# import numpy as np
import cvxpy as cp
import networkx as nx
import autograd.numpy as np
from autograd import grad

import pymanopt
import pymanopt.manifolds
import pymanopt.optimizers

from numpy.random import default_rng
from complex_elliptope import ComplexElliptope

from opt_utils import decompose_psd, normalize_rows, complex_hyperplane_rounding

In [2]:
def get_rand_complex(m, n, max_val=5):
    return np.round(np.random.rand(m, n) * max_val) + np.round(np.random.rand(m, n) * max_val) * 1j

# Phase recovery

In [3]:
n = 50  # number of observations
p = 20  # dimension of x
max_val = 10

In [4]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [5]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ cp.square(cp.real(u[i])) + cp.square(cp.imag(u[i])) == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

### solve SDP with cvxpy

In [6]:
U = cp.Variable((n,n), hermitian=True)
# U = cp.Variable((n,n), symmetric=True)  # for testing

In [7]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [8]:
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 1992.4382414640925
A solution U is [[ 1.        +0.00000000e+00j  0.00653328+3.76919323e-02j
   0.15665386-9.59591096e-03j ... -0.04894404+3.52883932e-01j
   0.17264479+1.52852258e-01j  0.09292542-1.51324065e-01j]
 [ 0.00653328-3.76919323e-02j  1.        +0.00000000e+00j
  -0.05364511+4.27207367e-04j ...  0.01336725-1.99520543e-01j
  -0.17512022-1.61800180e-01j -0.01714655+1.20581852e-01j]
 [ 0.15665386+9.59591096e-03j -0.05364511-4.27207367e-04j
   1.        +0.00000000e+00j ... -0.02934961+5.92599005e-02j
  -0.06872465+4.54208671e-02j  0.02211735-6.82634166e-03j]
 ...
 [-0.04894404-3.52883932e-01j  0.01336725+1.99520543e-01j
  -0.02934961-5.92599005e-02j ...  1.        +0.00000000e+00j
   0.18750914-3.92856892e-01j -0.20380959-2.66168528e-01j]
 [ 0.17264479-1.52852258e-01j -0.17512022+1.61800180e-01j
  -0.06872465-4.54208671e-02j ...  0.18750914+3.92856892e-01j
   1.        +0.00000000e+00j -0.02439734-4.98241600e-01j]
 [ 0.09292542+1.51324065e-01j -0.01714655-1.

In [9]:
opt_rank = np.linalg.matrix_rank(U.value, 1e-9)
print(opt_rank)

30


### Complex elliptope gradient

In [10]:
manifold = ComplexElliptope(n, opt_rank)

@pymanopt.function.autograd(manifold)
def manifold_cost(Y):
    return -np.real(np.trace(M @ Y @ np.conj(Y).T))

problem = pymanopt.Problem(manifold=manifold, cost=manifold_cost)
solver = pymanopt.optimizers.SteepestDescent(min_step_size=1e-9)
solution = solver.run(problem).point

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         -1.2008224979235397e+03    4.86800252e+02    
   2         -1.2020874647769517e+03    4.68404531e+02    
   3         -1.2023216997141515e+03    4.67997894e+02    
   4         -1.2023655397916875e+03    4.67922068e+02    
   5         -1.2023737612304872e+03    4.67907857e+02    
   6         -1.2023753035863651e+03    4.67905192e+02    
   7         -1.2023755929549666e+03    4.67904692e+02    
   8         -1.2023756472454684e+03    4.67904598e+02    
   9         -1.2023756574313204e+03    4.67904580e+02    
  10         -1.2023756593423661e+03    4.67904577e+02    
  11         -1.2023756597009119e+03    4.67904576e+02    
  12         -1.2023756597681815e+03    4.67904576e+02    
Terminated - min step_size reached after 12 iterations, 0.01 seconds.



In [12]:
problem.euclidean_gradient(U.value)

array([[ -80.59892507-3.42634841e-07j,   -0.52657529+3.03792956e+00j,
         -12.62613243-7.73420143e-01j, ...,
           3.9448364 +2.84420659e+01j,  -13.91498539+1.23197277e+01j,
          -7.4896886 -1.21965575e+01j],
       [  -0.79349158-4.57782817e+00j, -121.45380895+1.31559320e-08j,
           6.51540355+5.18860128e-02j, ...,
          -1.6235039 -2.42325300e+01j,   21.26901809-1.96512482e+01j,
           2.08251325+1.46451255e+01j],
       [  -2.19294149+1.34329290e-01j,    0.75095897-5.98025303e-03j,
         -13.99864774-2.46828731e-08j, ...,
           0.41085459+8.29558136e-01j,    0.96205215+6.35830246e-01j,
          -0.30961248-9.55594656e-02j],
       ...,
       [   8.39694901-6.05415597e+01j,   -2.29331603+3.42301927e+01j,
           5.03528528-1.01667617e+01j, ...,
        -171.56224859+6.24245011e-09j,  -32.16948966-6.73994112e+01j,
          34.96603038-4.56644708e+01j],
       [ -25.11492392-2.22356711e+01j,   25.47502777+2.35373394e+01j,
           9.99748767-

In [ ]:
problem.grad(solution)

In [ ]:
np.linalg.norm(problem.grad(solution))

In [ ]:
manifold.norm(np.zeros(solution.shape), problem.grad(decompose_psd(U.value)))

### Hyperplane rounding

In [ ]:
qp_cost = lambda u: -cp.real(cp.quad_form(u, M)).value

In [ ]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)